In [144]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [145]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [146]:
df= pd.read_csv('riceClassification.csv')

In [147]:
df.sample(3)

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
16544,16545,9050,163.187697,71.097956,0.900101,9236,107.344389,0.587357,385.411,0.765615,2.295252,0
5867,5868,6260,156.648670,52.583070,0.941978,6488,89.277542,0.743910,350.183,0.641496,2.979070,1
8113,8114,4270,130.944953,41.979872,0.947217,4414,73.734204,0.704272,289.233,0.641419,3.119232,1


In [148]:
df.drop('id',inplace=True,axis=1)

In [149]:
original_df =  df.copy()

In [150]:
X_train , x_temp ,y_train , y_temp = train_test_split(df.drop('Class',axis=1),df['Class'],test_size=.4,random_state=69)
X_valid,X_test,y_valid,y_test = train_test_split(x_temp,y_temp,test_size=.5,random_state=69)

In [151]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [152]:
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [153]:
# X_train_scaled = pd.DataFrame(X_train_scaled,columns=X_train.columns,index=X_train.index)
# X_valid_scaled = pd.DataFrame(X_valid_scaled,columns=X_valid.columns,index=X_valid.index)
# X_test_scaled = pd.DataFrame(X_test_scaled,columns=X_test.columns,index=X_test.index)


In [ ]:
class dataset(Dataset):
    def __init__(self,X,Y):
        self.X = torch.tensor(X,dtype=torch.float32).to(device)
        self.Y = torch.tensor(Y,dtype=torch.float32).to(device)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index] , self.Y[index]
        

In [182]:
training_data = dataset(X_train_scaled, y_train.values)
validation_data = dataset(X_valid_scaled, y_valid.values)
test_data = dataset(X_test_scaled, y_test.values)

In [188]:
train_dataloader = DataLoader(training_data,batch_size=8,shuffle=True)
validation_dataloader = DataLoader(validation_data,batch_size=8,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=8,shuffle=True)

In [197]:
HIDDEN_NEURON = 10 
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()
        self.input_layer =nn.Linear(df.drop('Class',axis=1).shape[1],HIDDEN_NEURON)
        self.linear = nn.Linear(HIDDEN_NEURON,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        x = self.input_layer(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [198]:
model = MyModel().to(device)

In [206]:
summary(model,(df.drop('Class',axis=1).shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [209]:
criterion =nn.BCELoss()
optimazer = Adam(model.parameters(),lr=1e-3)